### Visibility Weights

Main source: https://casaguides.nrao.edu/index.php/DataWeightsAndCombination

For optimal imaging performance it is critical that in a relative sense each visibility in the data have the correct weight after calibration -- that is, data with better sensitivity have more weight than data with less sensitivity. Formally, the post-calibration visibility weights should be equal to $1/\sigma_{ij}^2$ where $\sigma_{ij}$ is the rms noise of a given visibility.

$$\sigma_{ij} = \frac{2k}{\eta_{q} \eta_{c} A_{eff}} \sqrt{\frac{T_{sys, i} T_{sys, j}}{2 \Delta \nu_{ch} t_{dump}}}$$


Expected imaging rms noise:

$$\sigma_{image} = \frac{2k}{\eta_{q} \eta_{c} A_{eff}} \sqrt{\frac{T_{sys}^2}{N(N-1) \Delta \nu_{tot} t_{on} n_p}}$$

**In order to correctly combine and image data that have different $T_{sys}$, $\Delta \nu_{ch}$, $t_{ij}$, or antenna size it is essential to use visibility weights proportional to $\frac{1}{\sigma_{ij}^2}$.**

### Relative Overall Weights between Configurations

Additionally, when combining data from **different antenna configurations**, one will get optimal overall sensitivity to all spatial scales by matching the surface brightness sensitivity at each uv-distance. This can only be achieved by having *time-on-source per configuration in the right proportion*. This topic is covered in [ALMA Memo 598](http://library.nrao.edu/public/memos/alma/memo598.pdf). This memo informs the relative amount of time that ALMA observes a project with the 7m-array versus the 12m-array, and **compact versus extended 12m-array configurations**. 

However, since telescope time is expensive, one typically does not actually observe in the optimal proportion, in that case one will not fully realize the expected "impact" of adding the less sensitive configuration data. In that case, one may chose to "up-weight" the less sensitive configuration explicitly by changing its data weights, above and beyond $1/\sigma_{ij}^2$. However, it should be noted that nothing comes for free, and such a change will come at the expense of overall image sensitivity though it may very well be the optimal choice for your science case -- for example, if you are particularly interested in large scale structures you might apply extra up-weighting to the dataset(s) with the shortest baselines. **Finding the optimal up-weighting is a matter of experimentation**, but can easily be explored using the visweightscale parameter in concat. 

As a general rule of thumb extra up-weighting by more than **a factor of 4 or so is not recommended**. Before experimenting with this however, its always *best to start* with data that simply has the correct **$1/\sigma_{ij}^2$ weights**. Additionally, some degree of relative re-weighting can also be accomplished simply by using the *Briggs Robust parameter*, without explicitly changing the visibility weights, and generally it is also a good idea to try this first before more aggressive up-weighting is attempted.

### Weights in CASA

A memo describing weights in CASA, in particular the significant changes that were made with CASA 4.2.2, can be found at https://casa.nrao.edu/Memos/CASA-data-weights.pdf. 

- Different CASA version use different weight, especially before and after version 4.2.2; dangerous if we combine "product" of calibration from different CASA version.

- Simplest solution is to recalibrate with newer CASA version -> this is what we have done so far (so it is already using correct *weight*)

- Other solution is written in same page as above: https://casaguides.nrao.edu/index.php/DataWeightsAndCombination#What_Are_the_Options_for_Adjusting_the_Weights_for_Older_Reductions.3F


### Absolute Accuracy of the Data Weights

As mentioned several times in this guide, good imaging performance is only subject to correct relative weights, which is achieved as of CASA >4.2.2 provided <code>calwt=True</code> for the amplitude gain table. Even for Case 1 (4.2.1 and lower), the relative weights within a given dataset are reasonably accurate in most cases due to the $1/T_{sys}^{2}$ scaling. However, there are some instances, for example, **uv-model fitting** where one would like to use the visibility weights as an absolute measure of $1/\sigma^{2}$.

All PI data taken so far with ALMA are Hanning smoothed online, but it is up to the user at proposal creation time to decide if additional channel averaging is done in the correlator (typically to reduce the data rate). Data that are Hanning smoothed but not channel averaged by the ALMA online system have a theoretical effective channel width 2.667 times that reported by for example listobs from the point of view of calculating the sensitivity of a single channel (it is a factor of 2.0 for the FWHM, i.e. channel resolution). The CASA initialization of $2 \Delta \nu_{ch} t_{dump}$ does not currently take this into account -- CASA does not have a way of knowing presently what the online system did, only how the data appears on disk, though this will change with future versions of the online software. Thus, presently CASA weights for this case are in principle a factor of 2.667 times smaller than they should be from a sensitivity point of view. For online channel averaging of two or more channels, the difference between the channel width and the effective channel width diminishes rapidly (2x channel averaging has a factor 1.6, 4x a factor 1.2, 8x a factor of 1.1). Additionally, the current weights do not include the correlator and quantization efficiencies $\eta_{q}$ and $\eta_c$. These have values of about 0.88 and 0.96, respectively.

--- Starting in Cycle 3, the effective channel width will be recorded in the raw data and used by CASA to initialize the weights for CASA versions >=4.2.2.

Weight

$w_{ij} = \frac{1}{\sigma_{ij}}$

### Rough estimation

If we compare the rms of 2 images (from 2 MS).

$$\frac{\sigma'}{\sigma} = \sqrt{\frac{N(N-1) \Delta \nu_{tot} t_{on} n_p}{N'(N'-1) \Delta \nu'_{tot} t'_{on} n'_p}} \frac{\left<T'_{sys}\right>}{\left<T_{sys}\right>}$$

We can imagine they have different configuration, mode (TDM/FDM), Tsys, weather condition, antenna flagging, etc 



Inside each MS, they have weight "Wt", which calculated based on their parameter

When we concat, this weight inside MS that need (if needed) to be scaled

If we use current CASA version to re-calibrate the MS, the weight should be correct already. So rescaling of this weight, only needed because overlapping of uvdist bacause of different configuration? 

$$\frac{\sigma_{ij}}{\sigma} \propto \sqrt{\frac{N(N-1) \Delta \nu_{tot} t_{on}}{2 \Delta \nu_{ch} t_{dump}}} $$

$$\sigma_{ij} \propto \sigma \sqrt{\frac{N(N-1) \Delta \nu_{tot} t_{on}}{2 \Delta \nu_{ch} t_{dump}}}$$

$$\frac{\sigma'_{ij}}{\sigma_{ij}} \propto \frac{\left<T'_{sys}\right>}{\left<T_{sys}\right>} \sqrt{\frac{\Delta \nu_{ch} t_{dump}}{\Delta \nu'_{ch} t'_{dump}}} $$

OR 

$$\frac{\sigma'_{ij}}{\sigma_{ij}} \propto \frac{\sigma'}{\sigma} \sqrt{\frac{N'(N'-1) \Delta \nu'_{tot} t'_{on}}{N(N-1) \Delta \nu_{tot} t_{on}}} \sqrt{\frac{\Delta \nu_{ch} t_{dump}}{\Delta \nu'_{ch} t'_{dump}}} $$

Maybe we can use last equation?